<div class="alert alert-info">
Ссылка для просмотра ноутбука в интерактивном режиме для использования гиперссылок и корректного отображения разметки:<br>
<a href='https://nbviewer.org/github/yulianikola/portfolio/blob/master/parsing_python/habr_avast.ipynb'>habr_avast</a></div>

### Парсинг 
#### Сайты habr.ru и avast.com

<p id="0"> 
<ul type="square"><a href="#1"><li>Задание 1 (habr)</li></a>
<a href="#2"><li>Задание 2 (avast)</li></a>
<ul><a href="#2.1"><li>Парсинг</li></a>
<a href="#2.2"><li>Формирование датафрейма (вар. 1)</li></a>
<a href="#2.3"><li>Формирование датафрейма (вар. 2)</li></a></ul>

<p id="1"> 
<h4>Задание 1 (habr)</h4>

* Спарсить страницу со свежими новостям на habr.com/ru/all/.
* Необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. 
* Поиск вести по всей доступной preview-информации и всего текста статьи целиком.
* Сформировать датафрейм со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

Задаем адрес сайта и список ключевых слов для поиска:

In [2]:
url = 'https://habr.com/ru/all/'
keywords = ['python', 'аналитик', 'data science']

Создаем функцию, которая принимает адрес сайта и список ключевых слов и возвращает датафрейм со столбцами: дата - заголовок - ссылка - текст статьи.

In [4]:
def get_habr_news(url, keywords):  
    
    """
    Функция парсит страницу с новостями Хабра.
    Параметры: адрес страницы, список ключевых слов.
    Отбирает статьи на основе вхождения ключевых слов в превью, хабы, заголовок и текст статьи.
    Возвращает датафрейм со столбцами: дата - заголовок - ссылка - текст статьи.
    """
    
    req = requests.get(url)
    soup = BeautifulSoup(req.text, 'html.parser')
    
    # собираем посты с новостями
    posts = soup.find_all('article', class_ = 'tm-articles-list__item')
    
    habr_news = pd.DataFrame()
    
    for post in posts:

        # извлекаем id поста, если его нет, то, наверное, это что-то странное и пропускаем
        post_id = post.attrs.get('id')
        if not post_id:
            continue

        # достаем превью, может быть в коде в двух вариантах
        if post.find('div', class_='article-formatted-body article-formatted-body_version-1'):
            preview = post.find('div',
                                class_='article-formatted-body article-formatted-body_version-1').text.replace('\n\r\n','')
        else:
            preview = post.find('div', class_='article-formatted-body article-formatted-body_version-2').text
            
        # достаем заголовок, хабы
        title = post.find('a', class_='tm-article-snippet__title-link').text
        hubs = post.find_all('a', class_='tm-article-snippet__hubs-item-link')
        
        # достаем ссылку, дополняем до полной
        art_link = 'https://habr.com' + post.find('a', class_='tm-article-snippet__title-link').get('href')
        
        # переходим по ссылке статьи и извлекаем ее текст, может быть в коде в двух вариантах
        article_soup = BeautifulSoup(requests.get(art_link).text, 'html.parser')
        time.sleep(0.1)
        if article_soup.find('div', class_='article-formatted-body article-formatted-body_version-1'):
            art_text = article_soup.find('div', class_='article-formatted-body article-formatted-body_version-1').text
        else:
            art_text = article_soup.find('div', class_='article-formatted-body article-formatted-body_version-2').text

        # проверяем, есть ли ключевые слова в заголовке, превью, хабах поста или тексте статьи
        # если да, то записываем в переменные
        if any([(keyword in preview.lower()) or (keyword in title.lower()) or (keyword in art_text.lower()) or
                any([keyword in hub.text.lower() for hub in hubs]) 
                for keyword in keywords]):        

            # извлекаем дату в формате для datetime и переводим в объект datetime
            date =  pd.to_datetime(
                post.find('span', class_='tm-article-snippet__datetime-published').find('time').get('datetime'), dayfirst=True)

            # записываем извлеченную выше ссылку
            link = art_link

            # собираем полученные данные в словарик
            # дату представляем в удобном нам формате
            row = {'date': date.strftime('%d.%m.%y, %H:%M'), 'title': title.strip(), 'link': link, 'text': art_text}
            
            # записываем словарик в датафрейм
            habr_news = pd.concat([habr_news, pd.DataFrame([row])]) 
            
    habr_news = habr_news.reset_index().drop('index', axis = 1)
    return habr_news

Применяем функцию, получаем искомый датафрейм.

In [5]:
get_habr_news(url, keywords)

,date,title,link,text
0,"25.07.21, 16:46",Машинное обучение в Excel при помощи Python и ...,https://habr.com/ru/company/skillfactory/blog/...,К старту курса о машинном и глубоком обучении ...
1,"25.07.21, 15:21","Мой уход из Яндекса, как не потерять мотивацию...",https://habr.com/ru/post/569522/,Всем привет! Хотела бы поделиться своей истори...
2,"25.07.21, 14:50",А я у тебя точно первый? Или как выбрать (перв...,https://habr.com/ru/post/569518/,"Я часто вижу, как разработчики разного уровня ..."
3,"25.07.21, 12:05",Настройка рекламы в TikTok для продвижения инт...,https://habr.com/ru/post/569490/,На данный момент TikTok является самой популяр...
4,"25.07.21, 12:01",Сборка с Bazel в реальном проекте,https://habr.com/ru/company/ntechlab/blog/540728/,"Привет, Хабр.В этой статье я расскажу о практи..."
5,"25.07.21, 11:13",Дайджест интересных материалов для мобильного ...,https://habr.com/ru/company/productivity_insid...,В новом дайджесте тесты и танцы с ассетами в i...
6,"25.07.21, 10:49",Фреймворк на PHP: CodeIgniter 4. «Лёгкий приме...,https://habr.com/ru/post/569474/,"\n​ Copyright 2021, codeigniter.com: a project..."
7,"25.07.21, 09:56","Подсчёт ссылок не так прост, как кажется: опыт...",https://habr.com/ru/post/569450/,Подсчёт ссылок обычно предлагается как самый п...
8,"25.07.21, 09:18","UX/UI-дизайнер: все, что вы хотели знать, но б...",https://habr.com/ru/company/hays/blog/569434/,"На связи опять ребята из Hays, и сегодня мы ра..."


<h4><a href="#0">Наверх</a></h4>

<p id="2"> 
<h4>Задание 2 (avast)</h4>

* Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Check https://www.avast.com/hackcheck/.
* Список email задаем переменной в начале кода: EMAIL = [xxx@x.ru, yyy@y.com]
* Сформировать датафрейм со столбцами: <дата утечки> - <источник утечки> - <описание утечки>
* Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

<p id="2.1"> 
<h5>Парсинг</h5>

In [1]:
import json

In [2]:
email_list = ['xxx@x.ru', 'yyy@y.com']

В инструменте разработчика видно, что при запросе на проверку отправляется post запрос на другой сайт. Заполняем из раздела Request Payload и раздела Request Headers:

In [3]:
url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
payload = {'emailAddresses': email_list}
headers = {'Vaar-Version': '0', 'Vaar-Header-App-Product': 'hackcheck-web-avast', 'Vaar-Header-App-Product-Name': 'hackcheck-web-avast', 'Vaar-Header-App-Build-Version': '1.0.0' }

Отправляем post-запрос:

In [6]:
req_2 = requests.post(url, json = payload, headers = headers)
req_2

<Response [200]>

Ответ на запрос приходит в виде словаря, значит из него можно сделать датафрейм:

In [7]:
resp_dict = req_2.json()

Разберем структуру словаря, приходящего в ответ на запрос. <br>Ключи словаря:

In [8]:
resp_dict.keys()

dict_keys(['breaches', 'data', 'summary'])

Разберем значение по ключу 'breaches'. Значение по ключу 'breaches' - словарь, назовем его вложенный словарь (первого уровня).
Первая пара его значений:

In [12]:
list(resp_dict['breaches'].items())[0]

('2',
 {'breachId': 2,
  'site': 'linkedin.com',
  'recordsCount': 158591429,
  'description': "In 2012, online professional networking platform LinkedIn fell victim to a breach of its members' passwords. Four years later, 117 million email and password combinations from that breach appeared for sale on a dark web marketplace. \n\nThe leaked passwords had only been protected by unsalted SHA-1 cryptographic hashing, which prompted LinkedIn to enforce salted hashing after the breach. Russian national Yevgeniy Nikulin was accused of the breach and was extradited from the Czech Republic to the United States as of March 2018.",
  'publishDate': '2016-10-21T00:00:00Z',
  'statistics': {'usernames': 0, 'passwords': 111975337, 'emails': 158591429}})

Ключи вложенного словаря (первого уровня) - номера утечек:

In [13]:
resp_dict['breaches'].keys()

dict_keys(['2', '3', '12', '15', '41', '2961', '3164', '3176', '3520', '3587', '3669', '13094', '13662', '16768', '17009', '17110', '17670', '37177'])

Значения вложенного словаря (первого уровня) - вложенный словарь (второго уровня).
На примере первого значения:

In [14]:
list(resp_dict['breaches'].values())[0]

{'breachId': 2,
 'site': 'linkedin.com',
 'recordsCount': 158591429,
 'description': "In 2012, online professional networking platform LinkedIn fell victim to a breach of its members' passwords. Four years later, 117 million email and password combinations from that breach appeared for sale on a dark web marketplace. \n\nThe leaked passwords had only been protected by unsalted SHA-1 cryptographic hashing, which prompted LinkedIn to enforce salted hashing after the breach. Russian national Yevgeniy Nikulin was accused of the breach and was extradited from the Czech Republic to the United States as of March 2018.",
 'publishDate': '2016-10-21T00:00:00Z',
 'statistics': {'usernames': 0, 'passwords': 111975337, 'emails': 158591429}}

Среди ключей вложенного словаря (второго уровня) видим нужную информацию:
- под ключом 'site' хранится источник утечки
- под ключом 'description' хранится описание утечки
- под ключом 'publishDate' хранится дата утечки

In [15]:
list(resp_dict['breaches'].values())[0].keys()

dict_keys(['breachId', 'site', 'recordsCount', 'description', 'publishDate', 'statistics'])

Разберем значение по ключу 'data'. Значение по ключу 'data' - вложенный словарь (первого уровня).
Первая пара его значений:

In [16]:
list(resp_dict['data'].items())[0]

('parapa.mail.ru',
 {'xxx@x.ru': [{'breachId': 3176,
    'usernameBreached': True,
    'passwordBreached': True}]})

Ключи вложенного словаря (первого уровня) - сайты/источники утечек:

In [16]:
resp_dict['data'].keys()

dict_keys(['parapa.mail.ru', 'adobe.com', 'cdprojektred.com', 'azcentral.com', 'linkedin.com', 'dropbox.com', 'wishbone.io', 'youku.com', 'myheritage.com', 'rayli.com.cn', 'canva.com', 'globalreach.eu', 'cfire.mail.ru', 'vk.com', 'forums.vkmonline.com', 'netlog.com', 'imesh.com', 'zynga.com'])

Значения вложенного словаря (первого уровня) - вложенный словарь (второго уровня).
Ключи вложенного словаря (второго уровня) - мейлы из запроса на проверку.
В качестве значений - список словарей, в каждом из которых номер утечки и утечка логина/пароля Да/Нет.

In [17]:
list(resp_dict['data'].values())[0]

{'xxx@x.ru': [{'breachId': 3176,
   'usernameBreached': True,
   'passwordBreached': True}]}

Разберем значение по ключу 'summary'. Значение по ключу 'summary' - вложенный словарь (первого уровня):

In [18]:
resp_dict['summary']

{'xxx@x.ru': {'breaches': [3, 12, 15, 2961, 3164, 3176]},
 'yyy@y.com': {'breaches': [2,
   3,
   15,
   41,
   3520,
   3587,
   3669,
   13094,
   13662,
   16768,
   17009,
   17110,
   17670,
   37177]}}

Ключи вложенного словаря (первого уровня) - мейлы из запроса на проверку:

In [19]:
resp_dict['summary'].keys()

dict_keys(['xxx@x.ru', 'yyy@y.com'])

Значения вложенного словаря (первого уровня) - вложенный словарь (второго уровня).
В качестве значений вложенного словаря (второго уровня)- список номеров утечек:

In [20]:
list(resp_dict['summary'].items())[0]

('xxx@x.ru', {'breaches': [3, 12, 15, 2961, 3164, 3176]})

Сформируем датафрейм.

<h4><a href="#0">Наверх</a></h4>

<p id="2.2">
<h5>Формирование датафрейма (вар. 1)</h5>

Сформируем датафрейм с информацией о номере, сайте, дате и описании утечки.

Формируем датафрейм на основе вложенного словаря по ключу 'breaches'.
Ключи-номера утечек становятся столбцами, ключи вложенных словарей второго уровня становятся индексами:

In [21]:
pd.DataFrame(req_2.json()['breaches'])

,2,3,12,15,41,2961,3164,3176,3520,3587,3669,13094,13662,16768,17009,17110,17670,37177
breachId,2,3,12,15,41,2961,3164,3176,3520,3587,3669,13094,13662,16768,17009,17110,17670,37177
site,linkedin.com,adobe.com,vk.com,imesh.com,dropbox.com,cdprojektred.com,cfire.mail.ru,parapa.mail.ru,globalreach.eu,rayli.com.cn,youku.com,myheritage.com,netlog.com,canva.com,zynga.com,azcentral.com,wishbone.io,forums.vkmonline.com
recordsCount,158591429,152046506,110121799,50913222,68591031,1871459,12880232,5029003,921298,4844179,94244880,110041653,52904632,137434010,216159495,705538,37475476,825654
description,"In 2012, online professional networking platfo...","In October of 2013, criminals penetrated Adobe...",Popular Russian social networking platform VKo...,"In June 2016, a cache of over 51 million user ...",Cloud storage company Dropbox suffered a major...,"In March 2016, CDProjektRed.com.com's forum da...","In July and August of 2016, two criminals carr...","In July and August 2016, two criminals execute...","In 2016, Global Reach Technology's database wa...","On an unconfirmed date, Chinese gossip site Ra...",Youku is a large Chinese video content company...,"In October 2017, a customer database belonging...",Netlog (formerly known as Facebox and Bingbox)...,"In May 2019, graphic-design site Canva's datab...","In September 2019, the game developer Zynga wa...","At an unconfirmed date, online Arizona newspap...","In January 2020, the online poll website Wishb...","At an unconfirmed date, the Russian-language m..."
publishDate,2016-10-21T00:00:00Z,2016-10-21T00:00:00Z,2016-10-29T00:00:00Z,2016-10-23T00:00:00Z,2016-10-24T00:00:00Z,2017-01-31T00:00:00Z,2017-02-14T00:00:00Z,2017-02-14T00:00:00Z,2017-03-15T00:00:00Z,2017-03-01T00:00:00Z,2017-03-24T00:00:00Z,2017-11-04T00:00:00Z,2018-02-18T00:00:00Z,2019-06-13T00:00:00Z,2019-10-17T00:00:00Z,2020-01-03T00:00:00Z,2020-05-28T00:00:00Z,2021-02-11T00:00:00Z
statistics,"{'usernames': 0, 'passwords': 111975337, 'emai...","{'usernames': 0, 'passwords': 129430596, 'emai...","{'usernames': 0, 'passwords': 107425364, 'emai...","{'usernames': 50909015, 'passwords': 50913222,...","{'usernames': 0, 'passwords': 68591031, 'email...","{'usernames': 1871410, 'passwords': 1871457, '...","{'usernames': 12880232, 'passwords': 12880222,...","{'usernames': 5029000, 'passwords': 5029003, '...","{'usernames': 13492, 'passwords': 921298, 'ema...","{'usernames': 4841886, 'passwords': 4833694, '...","{'usernames': 0, 'passwords': 94238535, 'email...","{'usernames': 0, 'passwords': 110041647, 'emai...","{'usernames': 0, 'passwords': 52902665, 'email...","{'usernames': 137304104, 'passwords': 61348974...","{'usernames': 214648143, 'passwords': 13476279...","{'usernames': 0, 'passwords': 702971, 'emails'...","{'usernames': 31111127, 'passwords': 10648734,...","{'usernames': 825566, 'passwords': 825654, 'em..."


Транспонируем, чтобы номера утечек стали индексами, а ключи вложенного словаря - столбцами:

In [22]:
pd.DataFrame(req_2.json()['breaches']).T.head()

,breachId,site,recordsCount,description,publishDate,statistics
2,2,linkedin.com,158591429,"In 2012, online professional networking platfo...",2016-10-21T00:00:00Z,"{'usernames': 0, 'passwords': 111975337, 'emai..."
3,3,adobe.com,152046506,"In October of 2013, criminals penetrated Adobe...",2016-10-21T00:00:00Z,"{'usernames': 0, 'passwords': 129430596, 'emai..."
12,12,vk.com,110121799,Popular Russian social networking platform VKo...,2016-10-29T00:00:00Z,"{'usernames': 0, 'passwords': 107425364, 'emai..."
15,15,imesh.com,50913222,"In June 2016, a cache of over 51 million user ...",2016-10-23T00:00:00Z,"{'usernames': 50909015, 'passwords': 50913222,..."
41,41,dropbox.com,68591031,Cloud storage company Dropbox suffered a major...,2016-10-24T00:00:00Z,"{'usernames': 0, 'passwords': 68591031, 'email..."


Сбрасываем индекс, получаем датафрейм с порядковым индексом:

In [23]:
df = pd.DataFrame(req_2.json()['breaches']).T.reset_index()
df.head()

,index,breachId,site,recordsCount,description,publishDate,statistics
0,2,2,linkedin.com,158591429,"In 2012, online professional networking platfo...",2016-10-21T00:00:00Z,"{'usernames': 0, 'passwords': 111975337, 'emai..."
1,3,3,adobe.com,152046506,"In October of 2013, criminals penetrated Adobe...",2016-10-21T00:00:00Z,"{'usernames': 0, 'passwords': 129430596, 'emai..."
2,12,12,vk.com,110121799,Popular Russian social networking platform VKo...,2016-10-29T00:00:00Z,"{'usernames': 0, 'passwords': 107425364, 'emai..."
3,15,15,imesh.com,50913222,"In June 2016, a cache of over 51 million user ...",2016-10-23T00:00:00Z,"{'usernames': 50909015, 'passwords': 50913222,..."
4,41,41,dropbox.com,68591031,Cloud storage company Dropbox suffered a major...,2016-10-24T00:00:00Z,"{'usernames': 0, 'passwords': 68591031, 'email..."


Столбец с датой в удобном формате:

In [24]:
df['date'] = pd.to_datetime(df['publishDate'], dayfirst = True).dt.strftime('%d.%m.%Y')
df.head()

,index,breachId,site,recordsCount,description,publishDate,statistics,date
0,2,2,linkedin.com,158591429,"In 2012, online professional networking platfo...",2016-10-21T00:00:00Z,"{'usernames': 0, 'passwords': 111975337, 'emai...",21.10.2016
1,3,3,adobe.com,152046506,"In October of 2013, criminals penetrated Adobe...",2016-10-21T00:00:00Z,"{'usernames': 0, 'passwords': 129430596, 'emai...",21.10.2016
2,12,12,vk.com,110121799,Popular Russian social networking platform VKo...,2016-10-29T00:00:00Z,"{'usernames': 0, 'passwords': 107425364, 'emai...",29.10.2016
3,15,15,imesh.com,50913222,"In June 2016, a cache of over 51 million user ...",2016-10-23T00:00:00Z,"{'usernames': 50909015, 'passwords': 50913222,...",23.10.2016
4,41,41,dropbox.com,68591031,Cloud storage company Dropbox suffered a major...,2016-10-24T00:00:00Z,"{'usernames': 0, 'passwords': 68591031, 'email...",24.10.2016


Оставляем нужные столбцы:

In [25]:
df = df.loc[:,['date', 'site', 'description', 'breachId']]
df.head()

,date,site,description,breachId
0,21.10.2016,linkedin.com,"In 2012, online professional networking platfo...",2
1,21.10.2016,adobe.com,"In October of 2013, criminals penetrated Adobe...",3
2,29.10.2016,vk.com,Popular Russian social networking platform VKo...,12
3,23.10.2016,imesh.com,"In June 2016, a cache of over 51 million user ...",15
4,24.10.2016,dropbox.com,Cloud storage company Dropbox suffered a major...,41


Дополним датафрейм столбцом с запрошенными имейлами.

Нужно получить список мейлов в соответствующем нашему датафрейму порядке. 
Для этого отсортируем датафрейм по номеру утечек.
И отбросим столбец с номером утечки:

In [26]:
df = df.sort_values('breachId')
df.drop('breachId', inplace = True, axis = 1)

Связка номер утечки-мейл есть в словаре под ключом 'data'.
Упорядочим вложенный словарь по номеру утечек:

In [27]:
df_mail = dict(sorted(req_2.json()['data'].items(), key=lambda kv: list(kv[1].values())[0][0]['breachId'], reverse=False)) 

# kv[1] - вложенный словарь с ключом-мейлом
# list(kv[1].values()) - список значений вложенного словаря
# list(kv[1].values())[0][0]['breachId'] - добираемся до номера утечки

Извлекаем по порядку из словаря мейлы и добавляем в датафрейм столбец с мейлами:

In [28]:
df['email'] = [', '.join(el.keys()) for el in list(df_mail.values())]

# list(df_mail.values()) список вложенных словарей
# el.keys() мейлы, в объекте типа списка
# ', '.join(el.keys()) преобразование списка в строку через запятую

Итоговый датафрейм:

In [29]:
df

,date,site,description,email
0,21.10.2016,linkedin.com,"In 2012, online professional networking platfo...",yyy@y.com
1,21.10.2016,adobe.com,"In October of 2013, criminals penetrated Adobe...","xxx@x.ru, yyy@y.com"
2,29.10.2016,vk.com,Popular Russian social networking platform VKo...,xxx@x.ru
3,23.10.2016,imesh.com,"In June 2016, a cache of over 51 million user ...","xxx@x.ru, yyy@y.com"
4,24.10.2016,dropbox.com,Cloud storage company Dropbox suffered a major...,yyy@y.com
5,31.01.2017,cdprojektred.com,"In March 2016, CDProjektRed.com.com's forum da...",xxx@x.ru
6,14.02.2017,cfire.mail.ru,"In July and August of 2016, two criminals carr...",xxx@x.ru
7,14.02.2017,parapa.mail.ru,"In July and August 2016, two criminals execute...",xxx@x.ru
8,15.03.2017,globalreach.eu,"In 2016, Global Reach Technology's database wa...",yyy@y.com
9,01.03.2017,rayli.com.cn,"On an unconfirmed date, Chinese gossip site Ra...",yyy@y.com


<h4><a href="#0">Наверх</a></h4>

<p id="2.3"> 
<h5>Формирование датафрейма (вар. 2)</h5>

Сформируем датафрейм через извлечение информации из вложенных словарей в цикле.

In [30]:
def get_breach_info (emails, url, headers):
    """
    Функция запрашивает информацию об утечках для списка мейлов на сайте avast.
    Параметры: список мейлов, адрес страницы, заголовки.
    Извлекает нужную информацию об утечках из словаря, приходящего в ответ на запрос.
    Возвращает датафрейм со столбцами: мейл - дата - источник - описание.
    """
    payload = {'emailAddresses': emails}
    resp_dict = requests.post(url, json = payload, headers = headers).json()
    df2 = pd.DataFrame()
    for email in resp_dict['summary'].keys():
        for breach_id in resp_dict['summary'][email]['breaches']:
            site = resp_dict['breaches'][str(breach_id)]['site']
            date = resp_dict['breaches'][str(breach_id)]['publishDate']
            descr = resp_dict['breaches'][str(breach_id)]['description']
            row = {'email': email, 'date': date, 'site': site, 'description': descr}
            df2 = pd.concat([df2, pd.DataFrame([row])])

    df2 = df2.reset_index().drop('index', axis = 1)
    df2['date'] = pd.to_datetime(df2['date'], format = '%Y-%m-%dT%H:%M:%SZ').dt.strftime('%d.%m.%y') 
    return df2

Используем функцию и получаем итоговый датафрейм:

In [31]:
get_breach_info(email_list, url, headers)

,email,date,site,description
0,xxx@x.ru,21.10.16,adobe.com,"In October of 2013, criminals penetrated Adobe..."
1,xxx@x.ru,29.10.16,vk.com,Popular Russian social networking platform VKo...
2,xxx@x.ru,23.10.16,imesh.com,"In June 2016, a cache of over 51 million user ..."
3,xxx@x.ru,31.01.17,cdprojektred.com,"In March 2016, CDProjektRed.com.com's forum da..."
4,xxx@x.ru,14.02.17,cfire.mail.ru,"In July and August of 2016, two criminals carr..."
5,xxx@x.ru,14.02.17,parapa.mail.ru,"In July and August 2016, two criminals execute..."
6,yyy@y.com,21.10.16,linkedin.com,"In 2012, online professional networking platfo..."
7,yyy@y.com,21.10.16,adobe.com,"In October of 2013, criminals penetrated Adobe..."
8,yyy@y.com,23.10.16,imesh.com,"In June 2016, a cache of over 51 million user ..."
9,yyy@y.com,24.10.16,dropbox.com,Cloud storage company Dropbox suffered a major...


<h4><a href="#0">Наверх</a></h4>